# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
file = "../WeatherPy/output_data/cities.csv"

weather_df = pd.read_csv(file)
weather_df.head()

,City,Country,Temperature (F),Date,Humidity %,Cloud,Wind Speed,Lat,Lng
0,Busselton,AU,57.24,1613157827,89,0,11.54,-33.6500,115.3333
1,Lompoc,US,54.23,1613157828,62,20,17.27,34.6391,-120.4579
2,Nikolskoye,RU,0.68,1613157829,78,75,6.71,59.7035,30.7861
3,Adavere,EE,6.93,1613157830,85,90,3.44,58.7086,25.8994
4,Alofi,NU,78.80,1613157831,94,99,6.91,-19.0595,-169.9187


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Use the lat and lng columns as the locations variable
locations = weather_df[["Lat", "Lng"]]

# Use the humidity % column of the df as the weight variable
humidity = weather_df["Humidity %"]

In [24]:
# Generate the map
# From class discussion, it is important to set a center, and a zoom level to help make the map display properly
# Got the center and zoom_level values through trial and error
fig = gmaps.figure(center=(25.0, 15.0), zoom_level = 1.8)

# Generate a new layer on the map - Heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [39]:
# Per homework instructions, I will narrow down to cities with the following:
# Temp 80 > and < 70, Wind Speed < 10, Cloudiness = 0

# Start by dropping NA values
weather_df.dropna(inplace = True)

# Start by finding cities with the perfect temperature (Temp 80 > and < 70)
perfect_temperature = weather_df.loc[(weather_df["Temperature (F)"] <= 80) & (weather_df["Temperature (F)"] >= 70)]
perfect_temperature

,City,Country,Temperature (F),Date,Humidity %,Cloud,Wind Speed,Lat,Lng
4,Alofi,NU,78.80,1613157831,94,99,6.91,-19.0595,-169.9187
7,Cape Town,ZA,70.54,1613157542,83,20,19.57,-33.9258,18.4232
16,Port Alfred,ZA,73.00,1613157864,93,73,5.99,-33.5906,26.8910
23,Hermanus,ZA,72.00,1613157882,81,23,2.66,-34.4187,19.2345
24,Port Elizabeth,ZA,73.40,1613157885,94,75,20.71,-33.9180,25.5701
31,Puri,IN,71.60,1613157898,88,100,1.14,19.8000,85.8500
35,Mabaruma,GY,77.00,1613157905,88,75,13.80,8.2000,-59.7833
38,Kahului,US,72.01,1613157911,56,20,3.44,20.8947,-156.4700
42,Hilo,US,73.60,1613157784,50,1,2.17,19.7297,-155.0900
53,Mar del Plata,AR,70.18,1613157673,94,90,14.97,-38.0023,-57.5575


In [36]:
weather_df.dtypes

City                object
Country             object
Temperature (F)    float64
Date                 int64
Humidity %           int64
Cloud                int64
Wind Speed         float64
Lat                float64
Lng                float64
dtype: object

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
